In [2]:
from ultralytics import YOLO
from pathlib import Path
import yaml
import random
import shutil
from IPython.display import display, Markdown

WARNING ⚠️ torchvision==0.20 is incompatible with torch==2.6.
Run 'pip install torchvision==0.21' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation


In [ ]:
DATASET_PATH = Path(r"D:\NX Hackathon\Dataset\Human Detection")  # Change this to your dataset path
IMAGE_DIR = DATASET_PATH / "images"
LABEL_DIR = DATASET_PATH / "labels"
CONFIG_FILE = DATASET_PATH / "human_dataset.yaml"
WEIGHTS_PATH = "yolo11m.pt"  # Pretrained weights
EPOCHS = 10
IMG_SIZE = 640
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

In [4]:
def split_dataset():
    train_img_dir = IMAGE_DIR / "train"
    val_img_dir = IMAGE_DIR / "val"
    test_img_dir = IMAGE_DIR / "test"
    train_lbl_dir = LABEL_DIR / "train"
    val_lbl_dir = LABEL_DIR / "val"
    test_lbl_dir = LABEL_DIR / "test"
    
    for folder in [train_img_dir, val_img_dir, test_img_dir, train_lbl_dir, val_lbl_dir, test_lbl_dir]:
        folder.mkdir(parents=True, exist_ok=True)
    
    image_files = list(IMAGE_DIR.glob("*.jpg"))
    random.shuffle(image_files)
    
    train_split = int(len(image_files) * TRAIN_RATIO)
    val_split = max(1, int(len(image_files) * VAL_RATIO))  # Ensure at least 1 image in val
    test_split = max(1, len(image_files) - train_split - val_split)  # Ensure at least 1 image in test
    
    train_files = image_files[:train_split]
    val_files = image_files[train_split:train_split + val_split]
    test_files = image_files[train_split + val_split:]
    
    for img_path in train_files:
        shutil.move(img_path, train_img_dir / img_path.name)
        shutil.move(LABEL_DIR / f"{img_path.stem}.txt", train_lbl_dir / f"{img_path.stem}.txt")
    
    for img_path in val_files:
        shutil.move(img_path, val_img_dir / img_path.name)
        shutil.move(LABEL_DIR / f"{img_path.stem}.txt", val_lbl_dir / f"{img_path.stem}.txt")
    
    for img_path in test_files:
        shutil.move(img_path, test_img_dir / img_path.name)
        shutil.move(LABEL_DIR / f"{img_path.stem}.txt", test_lbl_dir / f"{img_path.stem}.txt")
    
    display(Markdown("✅ **Dataset split into train/val/test folders. Validation and test sets ensured to have at least one image.**"))

In [12]:
def create_dataset_config():
    dataset_config = {
        "path": str(DATASET_PATH),
        "train": str(IMAGE_DIR / "train"),
        "val": str(IMAGE_DIR / "val"),
        "test": str(IMAGE_DIR / "test"),
        "nc": 1,  # Number of classes (human detection)
        "names": ["human"]
    }
    
    with open(CONFIG_FILE, "w") as f:
        yaml.dump(dataset_config, f, default_flow_style=False)
    display(Markdown(f"✅ **Dataset config file created at:** `{CONFIG_FILE}`"))

In [13]:
def download_model():
    if not Path(WEIGHTS_PATH).exists():
        display(Markdown("🔄 **Downloading YOLOv11 pretrained model...**"))
        model = YOLO("yolo11m.pt")  # Automatically downloads if not present
        display(Markdown("✅ **Model downloaded!**"))
    else:
        display(Markdown("✅ **Pretrained model already exists. Skipping download.**"))

In [14]:
def train_yolov11():
    display(Markdown("🚀 **Starting YOLOv11 training...**"))
    model = YOLO(WEIGHTS_PATH)
    results = model.train(data=CONFIG_FILE, epochs=EPOCHS, imgsz=IMG_SIZE)
    display(Markdown("✅ **Training completed!**"))


In [21]:
split_dataset()
download_model()

✅ **Dataset split into train/val/test folders. Validation and test sets ensured to have at least one image.**

✅ **Pretrained model already exists. Skipping download.**

In [22]:
create_dataset_config()

✅ **Dataset config file created at:** `D:\NX Hackathon\Dataset\Violence\human_dataset.yaml`

In [24]:
train_yolov11()

🚀 **Starting YOLOv11 training...**

New https://pypi.org/project/ultralytics/8.3.80 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.75  Python-3.12.9 torch-2.6.0+cu128.nv CUDA:0 (NVIDIA GeForce RTX 5080, 16303MiB)
engine\trainer: task=detect, mode=train, model=yolo11m.pt, data=D:\NX Hackathon\Dataset\Violence\human_dataset.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

train: Scanning D:\NX Hackathon\Dataset\Violence\labels\train... 42932 images, 595 backgrounds, 0 corrupt: 100%|██████████| 42932/42932 [00:21<00:00, 1957.57it/s]


train: New cache created: D:\NX Hackathon\Dataset\Violence\labels\train.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 3646, len(boxes) = 43061. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Scanning D:\NX Hackathon\Dataset\Violence\labels\val... 5366 images, 65 backgrounds, 0 corrupt: 100%|██████████| 5366/5366 [00:03<00:00, 1395.95it/s]

...

                   all       5366       5400      0.952      0.915      0.972       0.73



5 epochs completed in 0.587 hours.
Optimizer stripped from runs\detect\train15\weights\last.pt, 40.5MB
Optimizer stripped from runs\detect\train15\weights\best.pt, 40.5MB

Validating runs\detect\train15\weights\best.pt...
Ultralytics 8.3.75  Python-3.12.9 torch-2.6.0+cu128.nv CUDA:0 (NVIDIA GeForce RTX 5080, 16303MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 168/168 [00:24<00:00,  6.82it/s]


                   all       5366       5400      0.952      0.915      0.972       0.73
Speed: 0.0ms preprocess, 2.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train15


✅ **Training completed!**

In [13]:
from ultralytics import YOLO

# Load trained model
model = YOLO(r"G:\Drive của tôi\FPT Uni\NX Hackathon\HumanDetection Model\Yolov11l_20Epoch\best.pt")

# Export model to ONNX format
model.export(format="onnx",opset=16)


Ultralytics 8.3.75  Python-3.12.9 torch-2.6.0+cu128.nv CPU (AMD Ryzen 7 9800X3D 8-Core Processor)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs

PyTorch: starting from 'G:\Drive ca ti\FPT Uni\NX Hackathon\HumanDetection Model\Yolov11l_20Epoch\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (48.8 MB)

ONNX: starting export with onnx 1.17.0 opset 16...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  1.8s, saved as 'G:\Drive ca ti\FPT Uni\NX Hackathon\HumanDetection Model\Yolov11l_20Epoch\best.onnx' (96.8 MB)

Export complete (2.3s)
Results saved to G:\Drive ca ti\FPT Uni\NX Hackathon\HumanDetection Model\Yolov11l_20Epoch
Predict:         yolo predict task=detect model=G:\Drive ca ti\FPT Uni\NX Hackathon\HumanDetection Model\Yolov11l_20Epoch\best.onnx imgsz=640  
Validate:        yolo val task=detect model=G:\Drive ca ti\FPT Uni\NX Hackathon\HumanDetection Model\Yolov11l_20Epoch\best.onnx imgsz=640 data=E

'G:\\Drive của tôi\\FPT Uni\\NX Hackathon\\HumanDetection Model\\Yolov11l_20Epoch\\best.onnx'